In [48]:
import requests
import json

def register_player(base_url, register_dto):
    url = f"{base_url}/player/register"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(register_dto))
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 204:
        raise Exception("User with the same username or email address already exists")
    elif response.status_code == 400:
        raise Exception("Bad Request")
    elif response.status_code == 500:
        raise Exception("Server Error")
    else:
        response.raise_for_status()

In [251]:
def login_player(base_url, login_dto):
    url = f"{base_url}/player/login"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(login_dto))
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 204:
        raise Exception("User credentials do not match")
    elif response.status_code == 500:
        raise Exception("Server Error")
    else:
        response.raise_for_status()

In [80]:
def create_single_player_game(base_url, username):
    url = f"{base_url}/game/createSinglePlayer"
    params = {'username': username}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [103]:
def join_existing_game(base_url, username):
    url = f"{base_url}/game/joinExistingGame"
    params = {'username': username}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 400:
        raise Exception(response.text)
    elif response.status_code == 404:
        raise Exception(response.text)
    else:
        response.raise_for_status()

In [175]:
def player_move(base_url, game_id, move_type, edge_or_node_index, player_no):
    url = f"{base_url}/game/playerMove"
    data = {
        "gameID": game_id,
        "moveType": move_type,
        "edgeOrNodeIndex": edge_or_node_index,
        "playerNo": player_no
    }
    response = requests.post(url, data=data)
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 204:
        raise Exception("It's not the player's turn")
    elif response.status_code == 205:
        raise Exception("Not enough resources")
    elif response.status_code == 206:
        raise Exception("Not an appropriate location")
    elif response.status_code == 207:
        raise Exception("Can't upgrade the settlement")
    elif response.status_code == 208:
        raise Exception("Can't roll the dice more than once for a turn")
    elif response.status_code == 209:
        raise Exception("Can't end turn without rolling a dice")
    elif response.status_code == 210:
        raise Exception("Game over")
    elif response.status_code == 404:
        raise Exception("Game not found")
    elif response.status_code == 500:
        raise Exception("Server Error")
    else:
        response.raise_for_status()

In [124]:
def delete_game(base_url, game_id):
    url = f"{base_url}/game/{game_id}"
    headers = {'Content-Type': 'application/json'}
    response = requests.delete(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 400:
        raise Exception("Bad request, 'id' was not provided")
    elif response.status_code == 500:
        raise Exception("Not found record with that id or Internal server error")
    else:
        response.raise_for_status()

In [206]:
def print_game_id(game):
    print(f"Game ID: {game['gameID']}")

def print_settlements(game):
    for settlement in game['gameboard']['settlements']:
        print(f"Settlement ID: {settlement['settlementID']}, Node Index: {settlement['nodeIndex']}, Player No: {settlement['playerNo']}, Settlement Level: {settlement['settlementLevel']}")

def print_roads(game):
    for road in game['gameboard']['roads']:
        print(f"Road ID: {road['roadID']}, Edge Index: {road['edgeIndex']}, Player No: {road['playerNo']}")

def print_tiles(game):
    for tile in game['gameboard']['tiles']:
        print(f"Tile ID: {tile['tileID']}, Tile Index: {tile['tileIndex']}, Tile Type: {tile['tileType']}, Tile Number: {tile['tileNumber']}")

def print_player_info(game, player_no):
    player_key = f"player{player_no}"
    if game[player_key] is not None:
        print(f"Player {player_no} ID: {game[player_key]['playerID']}, Username: {game[player_key]['username']}, Email: {game[player_key]['email']}")
    else:
        print(f"Player {player_no} is not in the game.")

def print_player_score(game, player_no):
    score_key = f"player{player_no}Score"
    print(f"Player {player_no} Score: {game[score_key]}")

In [220]:
def print_player_resources(game, player_no):
    if 'playerCardDeckList' in game:
        if len(game['playerCardDeckList']) >= player_no:  # Check if player_no is valid
            print(f"Player {player_no} Resources:")
            deck = game['playerCardDeckList'][player_no - 1]  # player_no - 1 because list indices start at 0
            for card in deck['cards']:
                print(f"{card['cardType']}: {card['cardCount']}")
        else:
            print(f"No resource information found for Player {player_no}.")
    else:
        print("No resources found.")

In [242]:
register_dto = {
    "username": "testuser",
    "email": "testuser@example.com",
    "password": "password"
}

try:
    player = register_player("http://localhost:8080", register_dto)
    print(player)
except Exception as e:
    print(str(e))

{'playerID': 3, 'username': 'testuser', 'email': 'testuser@example.com', 'password': '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8', 'sessionKey': None, 'resetKey': None}


In [258]:
login_dto = {
    "username": "testuser",
    "password": "password"
}

try:
    player = login_player("http://localhost:8080", login_dto)
    print(player)
except Exception as e:
    print(str(e))

{'playerID': 3, 'username': 'testuser', 'email': 'testuser@example.com', 'password': '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8', 'sessionKey': '63199d9a-6518-4f36-9335-2fc04ce06f75', 'resetKey': None}


In [259]:
try:
    game = create_single_player_game("http://localhost:8080", "testuser")
    game_id = game['gameID']
    print(f"Game ID: {game_id}")
    print(game)
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    print(f"Response body: {http_err.response.text}")
except Exception as err:
    print(f"An error occurred: {err}")

Game ID: 6
{'gameID': 6, 'gameboard': {'id': 6, 'settlements': [{'settlementID': 27, 'nodeIndex': 36, 'playerNo': 1, 'settlementLevel': 1}, {'settlementID': 28, 'nodeIndex': 20, 'playerNo': 2, 'settlementLevel': 1}, {'settlementID': 29, 'nodeIndex': 11, 'playerNo': 3, 'settlementLevel': 1}, {'settlementID': 30, 'nodeIndex': 53, 'playerNo': 4, 'settlementLevel': 1}], 'roads': [{'roadID': 42, 'edgeIndex': 45, 'playerNo': 1}, {'roadID': 43, 'edgeIndex': 32, 'playerNo': 2}, {'roadID': 44, 'edgeIndex': 18, 'playerNo': 3}, {'roadID': 45, 'edgeIndex': 70, 'playerNo': 4}], 'tiles': [{'tileID': 96, 'tileIndex': 0, 'tileType': 'MOUNTAINS', 'tileNumber': 12}, {'tileID': 97, 'tileIndex': 1, 'tileType': 'MOUNTAINS', 'tileNumber': 8}, {'tileID': 98, 'tileIndex': 2, 'tileType': 'PASTURES', 'tileNumber': 9}, {'tileID': 99, 'tileIndex': 3, 'tileType': 'PASTURES', 'tileNumber': 9}, {'tileID': 100, 'tileIndex': 4, 'tileType': 'FOREST', 'tileNumber': 10}, {'tileID': 101, 'tileIndex': 5, 'tileType': 'HILLS

In [260]:
try:
    game = join_existing_game("http://localhost:8080", "testuser")
    game_id = game['gameID']
    print(f"Game ID: {game_id}")
    print(game)
except Exception as e:
    print(str(e))

Game ID: 6
{'gameID': 6, 'gameboard': {'id': 6, 'settlements': [{'settlementID': 27, 'nodeIndex': 36, 'playerNo': 1, 'settlementLevel': 1}, {'settlementID': 28, 'nodeIndex': 20, 'playerNo': 2, 'settlementLevel': 1}, {'settlementID': 29, 'nodeIndex': 11, 'playerNo': 3, 'settlementLevel': 1}, {'settlementID': 30, 'nodeIndex': 53, 'playerNo': 4, 'settlementLevel': 1}], 'roads': [{'roadID': 42, 'edgeIndex': 45, 'playerNo': 1}, {'roadID': 43, 'edgeIndex': 32, 'playerNo': 2}, {'roadID': 44, 'edgeIndex': 18, 'playerNo': 3}, {'roadID': 45, 'edgeIndex': 70, 'playerNo': 4}], 'tiles': [{'tileID': 96, 'tileIndex': 0, 'tileType': 'MOUNTAINS', 'tileNumber': 12}, {'tileID': 97, 'tileIndex': 1, 'tileType': 'MOUNTAINS', 'tileNumber': 8}, {'tileID': 98, 'tileIndex': 2, 'tileType': 'PASTURES', 'tileNumber': 9}, {'tileID': 99, 'tileIndex': 3, 'tileType': 'PASTURES', 'tileNumber': 9}, {'tileID': 100, 'tileIndex': 4, 'tileType': 'FOREST', 'tileNumber': 10}, {'tileID': 101, 'tileIndex': 5, 'tileType': 'HILLS

In [254]:
# Delete game
try:
    print(f"Game ID: {game_id}")
    result = delete_game("http://localhost:8080", game_id)
    print(result)
except Exception as e:
    print(str(e))

Game ID: 3
True


In [256]:
# Roll the dice
print(f"Game ID: {game_id}")
try:
    game = player_move("http://localhost:8080", game_id, "rollDice", 0, 1)
    print(game)
except Exception as e:
    print(str(e))

Game ID: 3
Game not found


In [249]:
# Add a settlement
try:
    game = player_move("http://localhost:8080", game_id, "addSettlement", 0, 1)
    print(game)
except Exception as e:
    print(str(e))

It's not the player's turn


In [ ]:
# Add a road
try:
    game = player_move("http://localhost:8080", game_id, "addRoad", 0, 1)
    print(game)
except Exception as e:
    print(str(e))

In [ ]:
# Upgrade a settlement
try:
    game = player_move("http://localhost:8080", game_id, "upgradeSettlement", 0, 1)
    print(game)
except Exception as e:
    print(str(e))

In [248]:
# End the turn
try:
    game = player_move("http://localhost:8080", game_id, "endTurn", 0, 1)
    print(game)
except Exception as e:
    print(str(e))

{'gameID': 3, 'gameboard': {'id': 3, 'settlements': [{'settlementID': 11, 'nodeIndex': 38, 'playerNo': 1, 'settlementLevel': 1}, {'settlementID': 12, 'nodeIndex': 10, 'playerNo': 2, 'settlementLevel': 1}, {'settlementID': 13, 'nodeIndex': 26, 'playerNo': 3, 'settlementLevel': 1}, {'settlementID': 14, 'nodeIndex': 47, 'playerNo': 4, 'settlementLevel': 1}], 'roads': [{'roadID': 18, 'edgeIndex': 49, 'playerNo': 1}, {'roadID': 19, 'edgeIndex': 9, 'playerNo': 2}, {'roadID': 20, 'edgeIndex': 32, 'playerNo': 3}, {'roadID': 21, 'edgeIndex': 62, 'playerNo': 4}], 'tiles': [{'tileID': 39, 'tileIndex': 0, 'tileType': 'PASTURES', 'tileNumber': 5}, {'tileID': 40, 'tileIndex': 1, 'tileType': 'FOREST', 'tileNumber': 8}, {'tileID': 41, 'tileIndex': 2, 'tileType': 'FOREST', 'tileNumber': 2}, {'tileID': 42, 'tileIndex': 3, 'tileType': 'FOREST', 'tileNumber': 4}, {'tileID': 43, 'tileIndex': 4, 'tileType': 'FIELDS', 'tileNumber': 8}, {'tileID': 44, 'tileIndex': 5, 'tileType': 'PASTURES', 'tileNumber': 4}, 

In [181]:
print_game_id(game)

Game ID: 36


In [225]:
print_player_resources(game,1)

Player 1 Resources:
BRICK: 5
LUMBER: 3
ORE: 0
GRAIN: 2
WOOL: 1


In [228]:
print_settlements(game)

Settlement ID: 9, Node Index: 36, Player No: 1, Settlement Level: 1
Settlement ID: 10, Node Index: 1, Player No: 2, Settlement Level: 1
Settlement ID: 11, Node Index: 6, Player No: 3, Settlement Level: 1
Settlement ID: 12, Node Index: 49, Player No: 4, Settlement Level: 1


In [229]:
print_roads(game)

Road ID: 12, Edge Index: 45, Player No: 1
Road ID: 13, Edge Index: 3, Player No: 2
Road ID: 14, Edge Index: 9, Player No: 3
Road ID: 15, Edge Index: 70, Player No: 4


In [230]:
print_tiles(game)

Tile ID: 39, Tile Index: 0, Tile Type: FIELDS, Tile Number: 2
Tile ID: 40, Tile Index: 1, Tile Type: MOUNTAINS, Tile Number: 12
Tile ID: 41, Tile Index: 2, Tile Type: FOREST, Tile Number: 6
Tile ID: 42, Tile Index: 3, Tile Type: PASTURES, Tile Number: 11
Tile ID: 43, Tile Index: 4, Tile Type: FOREST, Tile Number: 8
Tile ID: 44, Tile Index: 5, Tile Type: PASTURES, Tile Number: 8
Tile ID: 45, Tile Index: 6, Tile Type: MOUNTAINS, Tile Number: 3
Tile ID: 46, Tile Index: 7, Tile Type: FIELDS, Tile Number: 11
Tile ID: 47, Tile Index: 8, Tile Type: FIELDS, Tile Number: 9
Tile ID: 48, Tile Index: 9, Tile Type: DESERT, Tile Number: 0
Tile ID: 49, Tile Index: 10, Tile Type: FIELDS, Tile Number: 3
Tile ID: 50, Tile Index: 11, Tile Type: MOUNTAINS, Tile Number: 9
Tile ID: 51, Tile Index: 12, Tile Type: PASTURES, Tile Number: 10
Tile ID: 52, Tile Index: 13, Tile Type: PASTURES, Tile Number: 4
Tile ID: 53, Tile Index: 14, Tile Type: HILLS, Tile Number: 6
Tile ID: 54, Tile Index: 15, Tile Type: HILLS

In [231]:
print_player_info(game, 1)

Player 1 ID: 3, Username: testuser, Email: testuser@example.com


In [232]:
print_player_score(game, 1)

Player 1 Score: 1
